In [0]:
%sql
-- Lenguaje del Notebook: SQL
-- VERSIÓN 1: Creación de Funciones SQL Personalizadas (UDFs)

-- 1. Establecer el Catálogo y Esquema de trabajo (Ajusta a tu entorno)
-- Asegúrate de que este es el mismo catálogo y esquema donde
-- se creó la tabla 'dq_rule_library'.
USE CATALOG workspace;
USE SCHEMA dq_framework;

-- ------------------------------------------------------------------------------------
-- UDF 1: is_valid_nif_es
-- ------------------------------------------------------------------------------------
CREATE OR REPLACE FUNCTION is_valid_nif_es(nif STRING)
RETURNS BOOLEAN
COMMENT 'Valida NIF/NIE español (formato y letra). Devuelve TRUE si es válido, FALSE si no.'
RETURN
  CASE
    -- 1. Comprobar NULO o longitud incorrecta
    WHEN nif IS NULL OR LENGTH(trim(nif)) != 9 THEN false
    ELSE
      -- 2. Usar TRY_CAST para manejar de forma segura los casos en que la parte numérica
      --    no sea un número (ej: "A1234567Z"). Si falla, TRY_CAST devuelve NULL.
      --    Cualquier operación con NULL (ej: NULL % 23) devuelve NULL.
      --    La comparación final (NULL = 'Z') devuelve NULL, que el CASE no considera 'true'.
      substring('TRWAGMYFPDXBNJZSQVHLCKE', 
        (
          TRY_CAST(
            -- 3. Normalizar NIEs (X->0, Y->1, Z->2)
            regexp_replace(
              regexp_replace(
                regexp_replace(upper(substring(trim(nif), 1, 8)), 'X', '0'),
              'Y', '1'),
            'Z', '2')
          AS INT) 
        % 23) + 1, -- 4. Calcular el índice de la letra (módulo 23)
      1) = upper(substring(trim(nif), 9, 1)) -- 5. Comparar la letra calculada con la letra real
  END;

-- ------------------------------------------------------------------------------------
-- (Añadir aquí cualquier otra UDF SQL que necesites en el futuro)
-- ------------------------------------------------------------------------------------
-- CREATE OR REPLACE FUNCTION ...
-- ...